In [1]:

#

import warnings

from langchain.agents import AgentType
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_community.utilities import SQLDatabase
from langchain_community.vectorstores import FAISS
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

from langchain_core.prompts import PromptTemplate

from utils.load_model import embeddings, model # Replace or use your model here
from langchain.agents import AgentType, Tool, initialize_agent, AgentExecutor, create_react_agent

from langchain_core.tools import tool

warnings.filterwarnings('ignore')

In [2]:
erc_stuff_prompt_override = """

Role: Experienced company representative, proficient at finding information pertinent to any employee 
queries in your employee policies.
Task: Your task is to respond to the User Query with detailed information using the ``context``  and 
``metadata`` provided below.
Tone: Respond to the user query in a professional, courteous, and helpful tone
Adhere strictly to the company's policies and guidelines, and ensure your response is informative, 
accurate, and formatted according to the provided template.
-----
Context: {context}
-----
Template for Response:
Answer: [Provide a clear and direct answer to the user's inquiry.]
Explanation: [Provide a detailed explanation of the company's policy or guideline, using plain language and avoiding jargon.]
Source: [Reference the source of the information, including document names and page numbers.]

----------------
Given the Context and Instructions above, answer the below User Query with as much detail as possible.
-----
User Query: '{query}'
-----

"""

In [3]:
# SQL Toolkit initializations
leave_db = SQLDatabase.from_uri("sqlite:///data/db/employee_leaves.db")
insurance_db = SQLDatabase.from_uri("sqlite:///data/db/health_insurance.db")
it_db = SQLDatabase.from_uri("sqlite:///data/db/IT_support.db")
#
leave_toolkit = SQLDatabaseToolkit(db=leave_db, llm=model)
ins_toolkit = SQLDatabaseToolkit(db=insurance_db, llm=model)
it_toolkit = SQLDatabaseToolkit(db=it_db, llm=model)
#
index_path = "./data/erc_docs"
# Run custom stuff chain enterprise_rag_chain (erc)
erc_document_variable_name = "context"
erc_document_prompt = PromptTemplate(input_variables=["page_content"], template="{page_content}")

In [4]:
@tool
def get_policy_details(query: str) -> str:
    """
    A function that retrieves a policy answer based on a query using a specified index and model.

    Parameters:
    query (str): The query string used to search for the policy answer.
    eid (Optional): The entity ID to filter the search by.
    e_name (Optional): The entity name to filter the search by.

    Returns:
    str: The policy answer retrieved based on the query.
    """
    print(f"INFORM--Inside get_policy_answer: {query}")
    #
    policy_db = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    #
    retriever = policy_db.as_retriever(search_type="mmr", search_kwargs={"fetch_k": 20, "k": 5})
    #
    docs = retriever.get_relevant_documents(query=query, verbose=False)
    #
    erc_prompt = PromptTemplate(template=erc_stuff_prompt_override, input_variables=["context", "query"])
    #
    llm_chain = LLMChain(llm=model, prompt=erc_prompt)
    #
    chain = StuffDocumentsChain(
        llm_chain=llm_chain, document_prompt=erc_document_prompt,
        document_variable_name=erc_document_variable_name,
    )
    result = chain.run(input_documents=docs, query=query)
    return result

In [5]:
leave_agent_executor = create_sql_agent(
    llm=model, toolkit=leave_toolkit, verbose=False,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs={"handle_parsing_errors": True}
)
insurance_agent_executor = create_sql_agent(
    llm=model, toolkit=ins_toolkit, verbose=False,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs={"handle_parsing_errors": True}
)
it_agent_executor = create_sql_agent(
    llm=model, toolkit=it_toolkit, verbose=False,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs={"handle_parsing_errors": True}
)
#

In [ ]:
tools = [
    Tool(
        name="Leave agent system",
        func=leave_agent_executor.run,
        description="useful for when you need to answer questions about the leaves of an employee",    
    ),
    Tool(
        name="Insurance agent system",
        func=insurance_agent_executor.run,
        description="useful for when you need to answer questions about insurance details of an employee",    
    ),
    Tool(
        name="IT Support system",
        func=it_agent_executor.run,
        description="useful for when you need to answer questions about IT tickets. Input should be a fully formed question.",   
    ),
    Tool(
        name="Get_Policy_Details",
        func=get_policy_details,
        description="A tool that returns textual information about any policies",
    ),
]

In [ ]:
# leave_agent_executor.run("How many leaves do emp_id = '38433' have available?")
# leave_agent_executor.run("give me total number of leaves available of all employees and deduct total leaves available for 39592 employee? give me details of each step point wise")

In [ ]:
#
# memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(
    tools=tools, llm=model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors = True
)

In [ ]:
agent.run("How many leaves do employee '39592' have? and divide with the balance insurance amount from total leaves of this employee. provide me each answer in new line.")

In [ ]:
agent.run("Give me 2 employee names who use Northwind Health Plus insurance and provide details on Women's health and cancer rights from policy documents")

### create_react_agent

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors = True)

In [ ]:
agent_executor.invoke({
    "input": """ Give me 2 employee names who use Northwind Health Plus insurance and provide details on Women's health and cancer rights from policy documents
      give me answer point wise"""
    })

### create_openai_tools_agent

In [6]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [7]:
tools = [
    Tool(
        name="Leave_Agent_System",
        func=leave_agent_executor.run,
        description="useful for when you need to answer questions about the leaves of an employee",    
    ),
    Tool(
        name="Insurance_Agent_System",
        func=insurance_agent_executor.run,
        description="useful for when you need to answer questions about insurance details of an employee",    
    ),
    Tool(
        name="IT_Support_system",
        func=it_agent_executor.run,
        description="useful for when you need to answer questions about IT tickets. Input should be a fully formed question.",   
    ),
    Tool(
        name="Get_Policy_Details",
        func=get_policy_details,
        description="A tool that returns textual information about any policies",
    ),
]

In [60]:

template = """
    You are an exceptionally helpful assistant dedicated to providing accurate answers to user queries. 
    You have access to only specific set of tools each designed to assist users effectively and answer to those queries which are related to given tools, here are the tools you have {tools}. 
    Also provide answers which were there in chat history if it is related.
    If you are unable to provide information realted to the user query from the given tools simply respond with "I don’t know", avoid making up answers. 
    Here is the input {input} for which you need to provide an answer"""

In [61]:
# prompt = PromptTemplate(template=template,input_variables=["input","tools"])

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory,ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [62]:
memory = ChatMessageHistory(messages=[])
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,  handle_parsing_errors = True, verbose=False)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [63]:
input = """ Give me employee details who has Northwind Health Plus insurance and has Zero balance 
    also provide details on Balance billing protection
      give me answer point wise"""
#
result = agent_with_chat_history.invoke({
    "input": input,
    "tools":tools},
    config={"configurable": {"session_id": "Santosh"}},
    )
#
# result = agent_executor.invoke({
#     "input": """Tell me a joke about CTS""",
#     "tools":tools
#     })
#
res = result['output']
res_val = res.split("\n\n")
for val in res_val:
    print(val)


INFORM--Inside get_policy_answer: Balance billing protection
I found the following information for you:
1. Employee details with Northwind Health Plus insurance and a zero balance:
   - Michael Brown (Emp_ID: 42962)
   - Jennifer Miller (Emp_ID: 38076)
   - Jessica White (Emp_ID: 38613)
2. Balance billing protection:
   - Balance billing protection is a feature of the Northwind Standard plan offered to Contoso Electronics employees, which protects them from unexpected costs when visiting in-network providers.
   - Balance billing occurs when a provider bills you for the difference between what they charge and the amount that your insurance plan covers. With balance billing protection, you are not responsible for this difference when using in-network providers.
   - This protection does not apply to out-of-network providers or certain services like cosmetic services and experimental procedures.
   - To ensure that balance billing protection applies, it is recommended to get preauthoriza

In [64]:
result

{'input': ' Give me employee details who has Northwind Health Plus insurance and has Zero balance \n    also provide details on Balance billing protection\n      give me answer point wise',
 'tools': [Tool(name='Leave_Agent_System', description='useful for when you need to answer questions about the leaves of an employee', func=<bound method Chain.run of AgentExecutor(name='SQL Agent Executor', agent=RunnableAgent(runnable=RunnableAssign(mapper={
    agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
  })
  | PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct 

In [74]:
result = agent_with_chat_history.invoke({
    "input": """What was the previous question I asked""",
    "tools":tools},
    config={"configurable": {"session_id": "Santosh"}},
    )
result

{'input': 'What was the previous question I asked',
 'tools': [Tool(name='Leave_Agent_System', description='useful for when you need to answer questions about the leaves of an employee', func=<bound method Chain.run of AgentExecutor(name='SQL Agent Executor', agent=RunnableAgent(runnable=RunnableAssign(mapper={
    agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
  })
  | PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "sql_db_query - Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.\nsql_db_schema - Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tabl